# Polymake and GAP in Julia

This workshop consists of three parts

* Introduction to Julia
* GAP in Julia
* Polymake in Julia

## Materials

Materials for this course can be found at

[github.com/sebasguts/IntroductionGAPJuliaPolymake](https://github.com/sebasguts/IntroductionJuliaPolymakeGAP)

I stole many ideas and code snippets from the following sources:

* [JuliaBox Tutorials](https://github.com/JuliaComputing/JuliaBoxTutorials)
* [GAP Tutorials](https://www.gap-system.org/Manuals/doc/tut/chap0.html)
* [Polymake Tutorials](https://polymake.org/doku.php/user_guide/intro_tutorial)

## Credits

The software presented here was developed by many people:
* JuliaLang Developers
* GAP Developers
* Polymake Developers

# Lesson 1: Introduction to Julia

In this lesson, we will learn
* Basic control flow in Julia
* Function call semantics and types
* Use of handy data structures

## Basic control flow in Julia

We look at the following example of adding all odd entries in an array `A`

In [1]:
A = 1:20
accum = 0
for i in A
    if i % 2 != 0
        accum += i
    end
end
accum

100

Lets have a look at the code

Arrays are created using `[]`, and you can create 1- or 2-dimensional arrays

In [2]:
A = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [3]:
B = [1 2;3 4]

2×2 Array{Int64,2}:
 1  2
 3  4

`for` loops use an `in` to access array or range elements
```julia
for i in A
```
and are ended by an `end`.

`if` clauses are also ended by an `end`, the other keywords needed for `if` clauses are `elseif` and `else`.

## Functions and types
Julia function syntax is pretty straight-forward

In [4]:
function oddsum(A)
    accum = 0
    for i in A
        if i % 2 != 0
            accum += i
        end
    end
    return accum
end

oddsum (generic function with 1 method)

In [5]:
oddsum(1:20)

100

Most of the time, it pays to look at Julias manual or package library. Using fancier Julia syntax and built-in functions, we can achieve the following

In [6]:
function oddSum2(A)
    return sum(i for i in A if isodd(i) )
end

oddSum2 (generic function with 1 method)

In [7]:
oddSum2(1:20)

100

### Types

Julia is a strictly typed language, i.e., every object has a fixed type

In [8]:
typeof(1)

Int64

In [9]:
typeof([1,2,3])

Array{Int64,1}

In [10]:
typeof([1 2;3 4])

Array{Int64,2}

There are abstract and concrete types in Julia, which can be used to define and dispatch functions.

In [11]:
foo(x::Int64) = x
foo(x::Array{Int64,1}) = x[1]

foo (generic function with 2 methods)

In [12]:
foo(1)

1

In [13]:
foo([1,2,3])

1

In [14]:
foo(1.)

MethodError: MethodError: no method matching foo(::Float64)
Closest candidates are:
  foo(!Matched::Array{Int64,1}) at In[11]:2
  foo(!Matched::Int64) at In[11]:1

Types are used in Julia to dispatch functions. There are abstract (used mostly for dispatch) and concrete types.

In [15]:
abstract type RockPaperScissors end
struct Rock <: RockPaperScissors end
struct Paper <: RockPaperScissors end
struct Scissors <: RockPaperScissors end

We can now see how Julia's multiple dispatch works

In [16]:
winner(::T, ::T) where T <: RockPaperScissors = :draw
winner(::Rock, ::Scissors) = :rock
winner(::Rock, ::Paper) = :paper
winner(::Scissors, ::Paper) = :scissors
# Make symmetric cases work
winner(x, y) = winner(y, x)

winner (generic function with 5 methods)

In [17]:
winner(Paper(),Rock())

:paper

We can even examine Julia's generated assembly to see that all generated functions are constant

In [18]:
@code_native winner(Rock(),Paper())

	.text
; ┌ @ In[16]:3 within `winner'
	movq	%rsi, -8(%rsp)
	movabsq	$140260894837392, %rax  # imm = 0x7F9108D00A90
	retq
; └


## Some data structures

In this section, we will learn about some built-in data structures in Julia

### Arrays

Arrays can be created using `[]` or the array constructor `Array{Type,Dim}`, and entries can be accessed using the `[]` operator

In [19]:
mat = Array{Int64,2}(undef,2,2)

2×2 Array{Int64,2}:
 140260931922192  140260931922256
 140260931922224  140260940568512

In [20]:
mat[1,1]

140260931922192

Submatrices can be easily accessed by giving ranges as incides

In [21]:
mat[1,1:2]

2-element Array{Int64,1}:
 140260931922192
 140260931922256

In [22]:
mat[:,2]

2-element Array{Int64,1}:
 140260931922256
 140260940568512

### Tuples

Tuples are lists in which each entry has a defined type

In [23]:
typeof((1,"foo"))

Tuple{Int64,String}

In Julia, they can be very handy for iteration and return values

In [24]:
foo() = (1,2)

foo (generic function with 3 methods)

In [25]:
(x,y) = foo()

(1, 2)

In [26]:
x

1

In [27]:
y

2

To generate a tuple with a single entry, we need to provide a `,` after the entry.

In [8]:
x = (1,)
typeof(x)

Tuple{Int64}

In [9]:
y = (1)
typeof(y)

Int64

### Dictionaries

Dictionaries are build-in hashmaps, that can be used to easily store and search for indexed objects

In [28]:
dict = Dict( :foo => 1, :bar => 2 )

Dict{Symbol,Int64} with 2 entries:
  :bar => 2
  :foo => 1

In [29]:
dict[:foo]

1

Using `Dict` for indexed sets is in most cases superior than using lists of tuples, as searching in `Dict`s is usually reasonably faster

### Generation of Data structures

As seen in the `sum` example above, the Julia generator syntax is often handy to generate data

In [12]:
f(x) = x^2
Tuple(f(i) for i in 1:10)

(1, 4, 9, 16, 25, 36, 49, 64, 81, 100)

This also works for array

In [14]:
[f(i) for i in 1:10 if i%2==0]

5-element Array{Int64,1}:
   4
  16
  36
  64
 100

We can also create $2$-dimensional arrays that way

In [15]:
[i*j for i in 1:5, j in 1:5 ]

5×5 Array{Int64,2}:
 1   2   3   4   5
 2   4   6   8  10
 3   6   9  12  15
 4   8  12  16  20
 5  10  15  20  25

or a large $1$-dim array

In [16]:
[ i * j for i in 1:5 for j in 1:5 ]

25-element Array{Int64,1}:
  1
  2
  3
  4
  5
  2
  4
  6
  8
 10
  3
  6
  9
 12
 15
  4
  8
 12
 16
 20
  5
 10
 15
 20
 25

We can also achieve a lot by using Julia's `.` syntax

In [13]:
f.(i for i in 1:10)

10-element Array{Int64,1}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

Also Dicts can be created this way

In [21]:
Dict(i => f(i) for i in 1:10)

Dict{Int64,Int64} with 10 entries:
  7  => 49
  4  => 16
  9  => 81
  10 => 100
  2  => 4
  3  => 9
  5  => 25
  8  => 64
  6  => 36
  1  => 1

## Structs

Custom Julia data structures are presented as structs

In [6]:
struct MyData
    len::Int64
    list::Array{Int64,1}
end

ErrorException: invalid redefinition of constant MyData

Structs are types and classes, of which objects can be created

In [31]:
mydata = MyData(5,[1,2,3])

MyData(5, [1, 2, 3])

Their entries can be accessed using `.`, and the names can be used for function defintions

In [32]:
length(x::MyData) = x.len

length (generic function with 1 method)

In [33]:
length(mydata)

5

One can also define custom constructors

In [4]:
function MyData(x::Int64)
    return MyData(x,Array{Int64,1}(undef,x))
end

MyData (generic function with 1 method)

In [5]:
MyData(5)

MethodError: MethodError: no method matching MyData(::Int64, ::Array{Int64,1})
Closest candidates are:
  MyData(::Int64) at In[4]:2